# A demo using Hydrogen Hamiltonian with GPT-QE.

In [14]:
import json, os, torch
from qwrapper.hamiltonian import compute_ground_state
from qwrapper.obs import PauliObservable
from gqe.mingpt.model import GPT
from gqe.mingpt.trainer import Trainer
from gqe.mingpt.utils import set_seed
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.mingpt.cost import EnergyCost
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor
from gqe.mingpt.layer import LayerWiseTrainer
from gqe.util import get_device
import logging

logging.getLogger('benchmark.molecule').setLevel(logging.WARNING)

nqubit = 12
n_gates = 10
num_layers = 6
iter = 200
n_sample = 50
seed = 3051
ignore_cache = True
distances = [2.5, 3.0]  # choices of the distance between two atoms
model_type = 'gpt2'
transformation = 'jordan-wigner'
is_bravyi = transformation == 'bravyi-kitaev'
MODEL_FILEBASE = '../saved_models/{}_model_beh2_sto3g_{}_{}_{}.json'
TRAJECTORY_FILEBASE = '../output/{}_trajectory_beh2_sto3g_{}_{}_{}_{}.txt'
ENERGY_FILEBASE = '../output/{}_energy_beh2_sto3g_{}_{}_{}.txt'
OTHER_FILEBASE = '../output/{}_other_beh2_sto3g_{}_{}_{}.json'

# Training

In [15]:

def find_ground_state_energy(distance, seed, ignore_cache=False):
    geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {distance}\n" + f"H 0.0 0.0 {2 * distance}\n"
    molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
    set_seed(seed)
    # prepare file
    model_output = MODEL_FILEBASE.format(model_type, str(distance), transformation, seed)
    other_output = OTHER_FILEBASE.format(model_type, str(distance), transformation, seed)

    if not ignore_cache and os.path.exists(model_output):
        return

    # prepare Hamiltonian
    hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=is_bravyi)

    ge = compute_ground_state(hamiltonian)
    print("ground state:", ge)

    # prepare operator_pool
    uccsd = UCCSD(nqubit, molecule)
    paulis = uccsd.paulis
    paulis.append(PauliObservable("IIIIIIIIIIII"))
    initializer = HFStateInitializer(n_electrons=4)
    print("hf state:", hamiltonian.exact_value(initializer.init_circuit(nqubit, [], "qulacs")))

    pool = DefaultOperatorPool(paulis)
    time_pool = [1 / (2 ** j) for j in range(2, 12)]
    time_pool.extend([-1 / (2 ** j) for j in range(2, 12)])
    cost = EnergyCost(hamiltonian, initializer, pool,
                      time_pool)

    def generate_model_config():
        model_config = GPT.get_default_config()
        model_config.model_type = model_type
        model_config.vocab_size = cost.vocab_size()
        model_config.block_size = cost.vocab_size()
        model_config.n_gates = n_gates  # The number of gates for each circuit
        model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
        model_config.embd_pdrop = 0.1
        model_config.resid_pdrop = 0.1
        model_config.attn_pdrop = 0.1
        model_config.energy_offset = 14
        return model_config

    # create a Trainer object
    def generate_train_config():
        train_config = Trainer.get_default_config()
        train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
        train_config.max_iters = iter
        train_config.num_workers = 10
        train_config.n_samples = n_sample
        return train_config

    trainer = LayerWiseTrainer(generate_train_config, generate_model_config, cost, num_layers, get_device())
    file_monitors = []
    for index in range(num_layers):
        file_monitor = FileMonitor()
        file_monitors.append(file_monitor)
        trainer.set_monitors(index, [PrintMonitor(), file_monitor])
    trainer.run()
    indices = []
    min_energy = None
    for index in range(num_layers):
        trajectory_output = TRAJECTORY_FILEBASE.format(model_output, str(distance), transformation, index, seed)
        file_monitors[index].save(trajectory_output)
        indices.extend(file_monitors[index].min_indices)
        min_energy = file_monitors[index].min_energy

    m = {"distance": distance,
         "exact_energy": ge,
         "computed_energy": min_energy,
         "computed_indices": indices,
         "n_gates": n_gates, "seed": seed}
    with open(other_output, "w") as f:
        f.write(json.dumps(m))


In [16]:
for d in distances:
    find_ground_state_energy(d, seed, ignore_cache=ignore_cache)

converged SCF energy = -15.1630689782417
ground state: -15.351491075144125
hf state: -15.163068978241704
layer: 1 starts running
number of parameters: 89.63M
running on device mps
iter_dt 0.00s; iter 0: train loss 0.40430 temperature: 5
mean_logits tensor([-15.0764, -14.7060, -14.6414, -14.5844, -14.8839, -15.0120, -14.9468,
        -15.1250, -14.8937, -14.9527, -15.0956, -14.7954, -14.7877, -14.9033,
        -14.9501, -15.1096, -15.0032, -14.7610, -15.0872, -15.0009, -14.9616,
        -15.0958, -14.8095, -14.8077, -15.0194, -14.8500, -15.1202, -15.1019,
        -14.7560, -15.1691, -15.0521, -14.9217, -15.0576, -14.8395, -15.0251,
        -14.7688, -15.1577, -14.8727, -14.9870, -14.6999, -15.1094, -14.6424,
        -14.9627, -15.0212, -14.7556, -14.9330, -15.0371, -14.9255, -14.7312,
        -14.9989], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-15.0589, -14.9218, -15.1181, -15.1304, -15.0992, -15.1298, -15.1468,
        -15.1558, -15.0858, -15.1436, -15.1609, -15.1809, 


KeyboardInterrupt



# Figure

In [ ]:
min_d = distances[0] - 0.1
max_d = distances[len(distances) - 1] + 0.1
n_bin = 100

xs = []
ys = []
ys3 = []
initializer = HFStateInitializer(n_electrons=4)
for j in range(n_bin):
    d = min_d + (max_d - min_d) / (n_bin - 1) * j
    geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {d}\n" + f"H 0.0 0.0 {2 * d}\n"
    molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
    hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)
    ge = compute_ground_state(hamiltonian)
    scf = hamiltonian.exact_value(initializer.init_circuit(nqubit, [], "qulacs"))
    xs.append(d)
    ys.append(ge)
    ys3.append(scf)


In [ ]:
xs2 = []
ys2 = []

for d in distances:
    xs2.append(d)
    with open(OTHER_FILEBASE.format(model_type, d, transformation, seed)) as f:
        ys2.append(json.loads(f.readline())['computed_energy'])

import matplotlib.pyplot as p

# p.grid('-')
p.plot(xs, ys, label='exact', linewidth=1, color='blue')
p.plot(xs2, ys2, label='computed', marker='x', linewidth=0, color='green')
p.plot(xs, ys3, label='hf', linewidth=1, color='gray')
# p.xlim([0.9, 2.3])
# p.ylim([-15.6, -15.2])
p.xlabel('bond length (angstrom)')
p.ylabel('energy value (Hartree)')
p.title('GPT-QE with BeH2 Hamiltonian (sto-3g basis, 12-qubits, 40-gates)')
p.legend()
p.show()